# Area statistics for protected areas in mainland Norway

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb) [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb)

**Author**: Willeke A'Campo

**Description:** This notebook is designed to compute area statistics for protected regions within mainland Norway. It specifically targets two datasets: **Verneområder** og **Foreslatte verneområder**, both provided by the Norwegian Environment Agency. In addition area statistics are calcualted for the whole of norway and for each regional zone. This allows for a comparative analysis beteween the protected areas and the overal Norwegian Territory. 

The attributes calculated can be divided into three groups:

1. Area variables for the protected areas:
    - Area (terrestrial and marine)
    - Perimeter (terrestrial and marine)
    - Land area (terrestrial)
    - Perimeter land area (terrestrial)

2. Overlay statistics for the protected area and the following datasets:
    - AR50 - Bonitet
        - Area proportion of land cover quality classes 
    - Bioklimatiske soner
        - Area proportion of bioclimatic zone classes
    - Infrastrukturindeks: 
        - Mean value of infrastructure index per protected area 
        - Area proportion of infrastructure index classes
    - Høydelag
        - Area proportion of elevation classes 

3. Spatial indices for the protected areas:
    - Density of protected area per 10x10km (SSB grid)
    - Shape Index for each protected area (land + marine)
    - Shape index for each protected area (land)






# Configure Notebook

In [1]:
# Variables to control the execution of the workflow
gis_server = True # True = use GIS server, False = use local files
load_data = False # True = load data from source, False = data is already loaded
test_area = False # True = test area, False = full area
prepare_data = False
prepare_duckdb = False # True = prepare duckdb, False = duckdb is already prepared
run_ar50 = False # True = run AR50 area calc , False = AR50 is already run
run_bioklima = False 
run_shape_index = False

# protected area
protected_area = "naturvernomrade"


if protected_area == "foreslatt_vern":
    protected_land = "f_vern_land"
    protected_sea = "f_vern_sjo"
    protected_land_sea = "f_vern_land_sjo"
    protected_land_vars = "f_vern_land_vars"
    
if protected_area == "naturvernomrade":
    protected_land = "vern_land"
    protected_sea = "vern_sjo"
    protected_land_sea = "vern_land_sjo"
    protected_land_vars = "vern_land_vars"

if protected_area == "norway":
    protected_land = "norge_land"
    protected_sea = "norge_sjo"
    protected_land_sea = "norge:land_sjo"
    protected_land_vars = "norge_land_vars"

In [2]:
import os 
from osgeo import gdal
from pathlib import Path
from itertools import islice

import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import box

# mapping libraries
# https://leafmap.org/faq/#how-to-use-a-specific-plotting-backend
import leafmap.foliumap as leafmap

In [3]:
project_path= Path.cwd().parents[0]
shell_path= os.path.join(project_path, "src", "shell_scripts")
python_path= os.path.join(project_path, "src", "py_scripts")

print(f"Gdal Version: {gdal.__version__} ")
print(f"Project Path: {project_path}")
print(f"Path to shell scripts: {shell_path}")
print(f"Path to python scripts: {python_path}")

Gdal Version: 3.8.4 
Project Path: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer
Path to shell scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/shell_scripts
Path to python scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/py_scripts


In [4]:
# import local python scripts
from py_scripts.my_ogr import *
from py_scripts.my_duckdb import *
from py_scripts.raster import * 
from py_scripts.config import load_catalog, load_parameters
from py_scripts import decorators as dec

# Data pre-processing

------------
**Data folder structure**

```bash
|-- data
|   |-- raw                                          # single raw files received or downloaded 
|   |-- interim
|   |    |-- vern_og_bevaring.db                     # DuckDB database with (pre-processed) data layers for spatial sql analysis (ca. 30 GB)
|   |    |-- vern_25833.gpkg                         # GPKG for viewing protected area data in GIS-software
|   |    |-- bevaring_25833.gpkg                     # GPKG for viewing environment data in GIS. 
|   |    |-- admin_25833.gpkg                        # GPKG for administrative data in GIS. 
|   |-- processed
|   |    |-- vern_og_bevaring.gpkg                   # GPKG with resulting data layers 
```
------------

<br>

**Data Pipeline**

1. Raw data is downloaded and stored into the `data\raw` folder. 
2. The raw files are converted (or if possible directly imported) into the GPKGs located in `data\interim`. These GPKGS are used to view the data in GIS-software before analysis. 
3. The data layers in the GPKG are loaded into a DuckDB database and/or a (geo)dataframe for spatial analysis. 
4. The results are exported to a GPKG located in `data\processed`. 
5. Raw and Interim files can be deleted by setting the variables: *delete_raw* and *delete_interim* to True



<br>

**GeoPackage information**

**INTERIM:**

/interim/*vern_25833.gpkg*

| Layer | Dataset Name | Description | Year | Source |
|-------| ------------ | ----------- | ---- | ------ |
| naturvernomrade | Naturvernområder| Nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/0) |
| foreslatt_vern | Foreslåtte naturvernområder | Planned nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/1)| 

- *naturvernomrade* and *foreslatt_vern* imported from arcgis rest server into GPKG using ogr2ogr

/interim/*bevaring_25833.gpkg*

|Layer| Dataset Name | Description | Year | Source |
|-----| ------------ | ----------- | ---- | ------ |
| ar50_flate | AR50 | Land cover classes | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard) |
| soner_2017_1km | Bioklimatiske soner | Bioclimatic zones stored in 1x1 km grid | 2017 | [Artsdatabanken](https://data.artsdatabanken.no/Natur_i_Norge/Natursystem/Beskrivelsessystem/Regional_naturvariasjon/Bioklimatisk_sone) |
| Infra25m | Infrastrukturindeks | Infrastructure index (25m) raster converted to vector datasett. | 2024 | Internal NINA datasett |

- *ar50_2022* stored on NINA's server. Manually loaded into GPKG
- *bioklima_soner_2017* received from V. Bakkestuen (NINA) manually loaded into GPKG. 
- *Infra25m* received from V. Bakkestuen (NINA) converted to vector and lmanually oaded into GPKG.

/interim/*admin_25833.gpkg*

| Filename | Dataset Name | Description | Year | Source |
|----------| ------------ | ----------- | ---- | ------ |
| fylke_2024 | Fylker, 2024 | Provincial Boundaries | 2024 | [SSB](https://www.ssb.no/en/kart/griddata) |
| norge_havareal_ar50 | Norge havareal | Norwegian sea area based on AR50 (Arealtype = hav) | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard)|
| norge_landareal_ar50 | Norge landareal | Norwegian land area based on AR50 (Arealtype =! hav) | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard)|
| norge_ruter_10km | SSB rutenett (10x10 km)| Grid for Norway | 2024 | [SSB](https://kart.ssb.no/) |
| region_2024 | Regioner, 2024 | Regional zones: Nord, Midt, Sør, Øst, Vest based on Provincial boundaries | 2024 |  [SSB](https://www.ssb.no/en/kart/griddata) |

**PROCESSED**

/processed/*vern_og_bevaring_25833.gpkg*

| Layer | Description | Year | Source |
|-------| ----------- | ---- | ------ |
| naturvernomrade | Nature protected areas containing attributes for (i) area statistics for  AR50, Bioclimatic zones, Infrastructureindex (ii) shape index. | 2024 | Project-folder, not published |
| foreslatt_vern | Planned nature protected areas containing attributes for (i) area statistics for  AR50, Bioclimatic zones, Infrastructureindex (ii) shape index. | 2024 | Project-folder, not published |
| fastland_norge | Norway mainland area containing attributes for area statistics for AR50, Bioclimatic zones, Infrastructureindex | 2024 | Project-folder, not published |
| region | Regional zones containing attributes for area statistics for AR50, Bioclimatic zones, Infrastructureindex | 2024 | Project-folder, not published |




In [5]:
# init catalog and params
catalog = load_catalog()
params = load_parameters()

# load paths from catalog
data_path = catalog["project_data"]["filepath"]
raw_data = os.path.join(data_path, "raw")
interim_data = os.path.join(data_path, "interim")

admin_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
vern_gpkg = os.path.join(data_path, "interim", "vern_25833.gpkg")
bevaring_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
vern_og_bevaring_gpkg = os.path.join(data_path, "interim", "vern_og_bevaring_25833.gpkg")
db_path = os.path.join(data_path, "processed", "bevaring.db")

print(f"Path to data: {data_path}")

Path to data: /home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data


## Download Data

### Vector Data
**Import data into `admin_25833.gpkg`**

Note Norwegians's Land Area is calculated from Norways National Map N50. Please check the *norge_landareal_n50* in GIS software to see if the layer is correctly clipped! 

In [6]:
if load_data:
    
    # load paths from catalog    
    fylke_path = catalog["fylke"]["filepath"]
    fylke_lyr = catalog["fylke"]["layer"]
    ar50_path = catalog["ar50"]["filepath"]
    ar50_lyr = catalog["ar50"]["layer"]
    n50_path = catalog["n50"]["filepath"]
    n50_lyr = catalog["n50"]["layer"]

    # RUTER 10KM NORGE (SSB)
    # download from ssb, unzip, convert to gpkg, remove shapefile and zip
    ! curl -o Ruter_10KM_norge.zip "https://www.ssb.no/natur-og-miljo/_attachment/375082?_ts=1685c0e69b8"
    ! unzip Ruter_10KM_norge.zip 
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln norge_ruter_10km -nlt MULTIPOLYGON $admin_gpkg $raw_data/"Ruter_10km_Norge.shp" -lco "OVERWRITE=YES"
    ! find $raw_data -type f -name "Ruter_10km_Norge.*" -exec echo "Removing {}" \; -exec rm {} \;
    ! rm Ruter_10KM_norge.zip
    
    # FYLKE
    ! ogrinfo "$fylke_path"
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln "$fylke_lyr" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$fylke_path" "$fylke_lyr"
    
    # NORGE
    # dissolve all fylke to one feature
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, 'Norge' AS fylkesnavn FROM $fylke_lyr" -t_srs EPSG:25833 -nln "norge" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$fylke_path"
    
    # AR50 HAVFLATE
    # import AR50 where artype = 82 and dissolve to one feature (Simplify by 1 m)
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, artype FROM $ar50_lyr WHERE artype = '82'" -t_srs EPSG:25833 -nln "norge_havflate_ar50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$ar50_path"
    
    # N50 HAVFLATE
    # where objtype = Havflate and dissolve to one feature
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, objtype FROM $n50_lyr WHERE objtype = 'Havflate'" -t_srs EPSG:25833 -nln "norge_havflate_n50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$n50_path"
    
    # N50 LANDAREAL
    remake = False
    
    if remake:
        # norge_landareal_n50: norge - norge_havflate_n50
        ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Difference(norge.geom, norge_havflate_n50.geom) AS geom, 'land' AS objtype FROM norge, norge_havflate_n50" \
            -t_srs EPSG:25833 -nln "norge_landareal_n50" -nlt MULTIPOLYGON admin_25833.gpkg admin_25833.gpkg
    else: 
        # copy layer from raw data
        ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln "norge_landareal_n50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$raw_data/N50_2024_hav_landareal_25833.gpkg" "norge_landareal_n50"

**Import data into `vern_25833.gpkg`**

In [7]:
if load_data: 
    # download data from ArcGIS REST directly into the interim folder
    os.chdir(os.path.join(data_path, "interim"))
    print(os.getcwd())
        
    # ArcGIS REST: Naturvern (02.2024)
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln naturvernomrade -nlt MULTIPOLYGON vern_25833.gpkg \
        "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/0/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

    # ArcGIS REST: Foreslåtte verneområder (02.2024)
    ! ogr2ogr -append -f "GPKG" -t_srs EPSG:25833 -nln foreslatt_vern -nlt MULTIPOLYGON vern_25833.gpkg \
        "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/4/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

**Import data into bevaring_25833.gpkg**

In [8]:
if True:
    os.chdir(os.path.join(data_path, "interim"))
    print(os.getcwd())
    
    bioklima_path = catalog["bioclimatic_zones"]["filepath"]
    bioklima_lyr = catalog["bioclimatic_zones"]["layer"]
    ar50_path = catalog["ar50"]["filepath"]
    ar50_lyr = catalog["ar50"]["layer"]

    print(ar50_path, ar50_lyr)
    
    # Copy GDB layers from NINA R to gpkg
    ! ogrinfo "$bioklima_path"
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln soner_2017_1km -nlt MULTIPOLYGON bevaring_25833.gpkg "$bioklima_path" "$bioklima_lyr"
    ! ogrinfo "$ar50_path"
    ! ogr2ogr -append -f "GPKG" -t_srs EPSG:25833 -nln "$ar50_lyr" -nlt MULTIPOLYGON bevaring_25833.gpkg "$ar50_path" "$ar50_lyr"


/data/scratch/wilaca/vern_og_bevaring/data/interim
${NINA_R}/GeoSpatialData/LandCover/Norway_LandResource_AR50/Original/versjon20221321/Arealressurskart - AR50/0000_25833_ar50_gdb.gdb ar50_flate
INFO: Open of `/home/NINA.NO/willeke.acampo/Mounts/R/GeoSpatialData/BiogeographicalRegions/Norway_PCA_klima/Original/20170614_Bioklima/20170614_Bioklima.gdb'
      using driver `OpenFileGDB' successful.
Layer: Soner2017 (Multi Polygon)
Layer: Seksjoner2017 (Multi Polygon)
INFO: Open of `/home/NINA.NO/willeke.acampo/Mounts/R/GeoSpatialData/LandCover/Norway_LandResource_AR50/Original/versjon20221321/Arealressurskart - AR50/0000_25833_ar50_gdb.gdb'
      using driver `OpenFileGDB' successful.
Layer: ar50_flate (Multi Polygon)
Warning 1: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of

### Raster Data



In [9]:
if load_data:
    infra_path = catalog["infra_25m"]["filepath"]
    
    # copy to project raw folder and convert to 25833
    ! gdalwarp -t_srs EPSG:25833 $infra_path $raw_data/infra_25m_25833.tif

In [10]:
# clip raster by polygon 
if load_data:
    ! gdalwarp -cutline "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/norge_landareal_n50.shp" -crop_to_cutline \
        $raw_data/infra_25m_25833.tif \
        $interim_data/infra_25m_25833_clip.tif

In [11]:
# WARNING: run in bash shell, otherise correct path to gdal_calc.py is not found. 
# multiply all cells by 1000 except for nodata values. 
if load_data:
    ! gdal_calc.py -A "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_clip.tif" \
        --outfile="//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000.tif" \
            --calc="numpy.where(A!=3.4e+38, A*1000, 3.4e+38)" --type=Float32 --NoDataValue=3.4e+38

In [12]:
# to integer
if load_data:
    ! gdal_translate -ot Int16 -of GTiff \
        "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000.tif" \
        "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000_int.tif"

In [13]:
# clip by coast again 
if load_data:
    ! gdalwarp -cutline "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/norge_landareal_n50.shp" -crop_to_cutline \
        $interim_data/infra_25m_25833_x1000_int.tif \
        $interim_data/infra_25m_25833_x1000int_coastalclip.tif 

In [14]:
# raster to vector
# to BIG! 
if load_data:
    ! gdal_polygonize.py $interim_data/infra_25m_25833_x1000_int.tif -f "GeoJSON" $raw_data/infra_25m_25833_x1000_int.geojson

**Write Metadata to .txt**

In [15]:
if load_data: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)

    path = os.path.join(data_path, "interim")

    ! gdalinfo --version
    ! chmod +x gdal_gpkg-info.sh
    ! ./gdal_gpkg-info.sh $path "vern_25833.gpkg"
    ! ./gdal_gpkg-info.sh $path "bevaring_25833.gpkg"
    ! ./gdal_gpkg-info.sh $path "admin_25833.gpkg"

## Data Preparation

### Sudy Area | Norway Mainland

This workflow is created and run for Norway mainland. If you like to test or use the workflow we recommend running it first on a study area. Below are bounding box coordinates provided for Dovrefjell, Fosen or Trondheim area. 

**Create Bounding Box** 

In [16]:
# bounding box Dovrefjell
bounding_box = "fosen"  # "dovrefjell" or "trondheim"

if bounding_box == "dovrefjell":
    xmin, ymin, xmax, ymax = 160000.00, 6900000.00, 260000.00, 6950000.00
if bounding_box == "fosen":
    xmin, ymin, xmax, ymax = 180000.00, 7010000.00, 290000.00, 7150000.00
if bounding_box == "trondheim":
    xmin, ymin, xmax, ymax = 260520.12, 7032142.5, 278587.56, 7045245.27

# Create a bounding box
boxBB =  box(xmin, ymin, xmax, ymax)
crs = 'EPSG:25833'

gdf_BB = gpd.GeoDataFrame(geometry=[boxBB])
gdf_BB['name'] = 'Dovre_BB'
gdf_BB.crs = crs
bounds = gdf_BB.bounds.to_numpy().tolist()[0]

In [17]:
if load_data: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)
    for file in ["vern", "bevaring", "admin"]:
        input_file = os.path.join(data_path, "interim", f"{file}_25833.gpkg")
        output_file = os.path.join(data_path, "tmp", f"{file}_25833_bbox.gpkg")
        
        #for layer_name in layer_names:
        ! chmod +x gdal_copy-file-bbox.sh
        ! ./gdal_copy-file-bbox.sh {input_file} {output_file} {xmin} {ymin} {xmax} {ymax}

In [18]:
if load_data: 
    # create a temporary test gpkg
    tmp_gpkg = os.path.join(data_path, "tmp", "tmp.gpkg")
    in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
    print(tmp_gpkg)

    sql_query = f'SELECT * FROM "ar50_flate" LIMIT 5'
    path = os.path.join(data_path, "tmp")
    ! ogr2ogr -f "GPKG" -nln "ar50_flate" -sql "$sql_query" {tmp_gpkg} {in_gpkg}

### AR50 - Bonitet

Translate the Bonitet classes using a lookup table.
 
&emsp; $Bonitet = (artype, artreslag, arskogbon, arjordbr, ardyrkning, arveget)$ 

In [19]:
# ar50 layer
layer_name, new_field_name = "ar50_flate", "ar50_bonitet"

if test_area:
    # import gpkg into ogr object 
    # add field name if it does not exist
    tmp_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
    ds, lyr = import_gpkg(tmp_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)
else:
    # import gpkg into ogr object 
    # add field name if it does not exist
    in_gpkg = os.path.join(interim_data, "bevaring_25833.gpkg")
    print(in_gpkg)
    ds, lyr = import_gpkg(in_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)

# Convert the first 5 features of the layer to a DataFrame
# use islice to limit the number of features to 5, to reduce computation time
df_AR50 = pd.DataFrame(feature.items() for feature in islice(lyr, 5))

# Print the DataFrame
df_AR50

/home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/bevaring_25833.gpkg


,objtype,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,arkartstd,målemetode,informasjon,oppdateringsdato,originaldatavert,navnerom,kopidato,områdeid,lokalid,fylkeid,komid,ar50_bonitet
0,arealressursflate,60,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,1,50,5022,None
1,arealressursflate,60,11,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,2,50,5022,None
2,arealressursflate,60,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,3,50,5022,None
3,arealressursflate,60,11,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,4,50,5022,None
4,arealressursflate,30,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,5,50,5022,None


In [20]:
# lookup table 
lookup_csv = os.path.join(data_path, "AR50_bonitet_lookup.csv")
lookup_df = pd.read_csv(lookup_csv)

# rename cols to correspond with AR50 lyr
lookup_df.rename(
    columns={
        "ARTYPE kode": "artype",
        "ARTRESLAG kode": "artreslag",
        "ARSKOGBON kode": "arskogbon",
        "ARJORDBR kode": "arjordbr",
        "ARDYRKING kode": "ardyrking",
        "ARVEGET kode": "arveget",
        "Bonitet kode": "ar50_bonitet",
    },
    inplace=True,
)

# reorder cols to correspond with AR50 lyr
lookup_df = lookup_df[
    [
        "artype",
        "arskogbon",
        "artreslag",
        "arveget",
        "arjordbr",
        "ardyrking",
        "Beskrivelse",
        "ar50_bonitet",
    ]
]

display(lookup_df.head(11))

,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,Beskrivelse,ar50_bonitet
0,10,98,98,98,98,98,Bebygd og samferdsel,14
1,20,98,98,98,24,98,Fulldyrka og Overflatedyrka jord,1
2,20,98,98,98,25,82,"Innmarksbeite, dyrkbar jord",2
3,20,98,98,98,25,81,Innmarksbeite,2
4,20,98,98,98,99,99,"Jordbruksareal, uspesifisert",1
5,30,11,31,98,98,82,"Barskog, impediment, dyrkbar jord",6
6,30,11,31,98,98,81,"Barskog, impediment",6
7,30,12,31,98,98,82,"Barskog, lav skogbonitet, dyrkbar jord",5
8,30,12,31,98,98,81,"Barskog, lav skogbonitet",5
9,30,13,31,98,98,82,"Barskog, middels skogbonitet, dyrkbar jord",4


In [21]:
# create lookup dict
# keys must be in same order as gpkg lyr fields
keys = (
    "artype",
    "arskogbon",
    "artreslag",
    "arveget",
    "arjordbr",
    "ardyrking",
    )

value= "ar50_bonitet"

lookup_dict = create_lookup_dict(
    lookup_df,
    keys=keys,
    value=value
)

# print first two entries of dict
print({k: lookup_dict[k] for k in list(lookup_dict)[:11]})

{(10, 98, 98, 98, 98, 98): 14, (20, 98, 98, 98, 24, 98): 1, (20, 98, 98, 98, 25, 82): 2, (20, 98, 98, 98, 25, 81): 2, (20, 98, 98, 98, 99, 99): 1, (30, 11, 31, 98, 98, 82): 6, (30, 11, 31, 98, 98, 81): 6, (30, 12, 31, 98, 98, 82): 5, (30, 12, 31, 98, 98, 81): 5, (30, 13, 31, 98, 98, 82): 4, (30, 13, 31, 98, 98, 81): 4}


In [22]:
if prepare_data:
    # loop through the features and reclassify the attribute value "ar50_bonitet"
    features_to_update = []

    for feature in lyr:
        # get the attribute values
        artype = feature.GetField("artype")
        arskogbon = feature.GetField("arskogbon")
        artreslag = feature.GetField("artreslag")
        arveget = feature.GetField("arveget")
        arjordbr = feature.GetField("arjordbr")
        ardyrking = feature.GetField("ardyrking")

        key = (int(artype), int(arskogbon), int(artreslag), int(arveget), int(arjordbr), int(ardyrking))
        if key in lookup_dict:
            new_value = lookup_dict[key]
            feature.SetField("ar50_bonitet", new_value)
            features_to_update.append(feature)

    print(f"Number of features to update: {len(features_to_update)}")
    # Batch update features
    for feature in features_to_update:
        lyr.SetFeature(feature)

    print("Finished updating Bonitet.")

    df_AR50_bon = pd.DataFrame(feature.items() for feature in islice(lyr, 5))
    display(df_AR50_bon.head())

# close OGR object
ds = None

In [23]:
if prepare_data:
    # Define the SQL queries
    sql_marine = f"SELECT * FROM ar50_flate WHERE ar50_bonitet = 17"
    sql_terrestisk = f"SELECT * FROM ar50_flate WHERE ar50_bonitet != 17"

    if test_area:
        # import gpkg into ogr object 
        # add field name if it does not exist
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        out_gpkg = os.path.join(data_path, "tmp", "interim_25833_bbox.gpkg")
    else:
        # import gpkg into ogr object 
        # add field name if it does not exist
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        out_gpkg = os.path.join(data_path, "interim", "interim_25833.gpkg")

    print(out_gpkg)

    # Create the ar50_marine layer
    ! ogr2ogr -f "GPKG" -nln "ar50_marine" -sql "$sql_marine" -append {out_gpkg} {in_gpkg}
    # Create the ar50_terrestisk layer
    ! ogr2ogr -f "GPKG" -nln "ar50_terrestisk" -sql "$sql_terrestisk" -append {out_gpkg} {in_gpkg}

### Infrastructure index

- Calculate and display the distribution of the infrastructure index for the whole of norway 
- Calculate and display the distribution of the infrastructure index per region
- Calculate and display the distribution of the infrastructure index per protected area
- Define the infrastructure index classes

### Topographic height

Classify the topographic height into 4 classes:
- 0-300m
- 301-600m
- 601-900m
- over 900m

## Load data into Spatial Database (DuckDB)


### Load data into DuckDB

In [24]:
if test_area:
    db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
else:
    db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")

**Load Bioklima soner into DuckDB**

In [25]:
if prepare_duckdb:
    # AR50 into duckdb
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "soner_2017_1km")
    else:
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "soner_2017_1km")

**Load AR50 into DuckDB**

In [26]:
# run time: 
if prepare_duckdb:
    # AR50 into duckdb
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate_msk")
    else:
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate")

**Load Protected Areas into DuckDB**

In [27]:
# import protected areas into DuckDB

# if Test is True, import test gpkg otherwise import the original gpkg
if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "vern_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
        load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")
    else:
        in_gpkg = os.path.join(data_path, "interim", "vern_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
        load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")

**Load Regions into DuckDB**

In [28]:
if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "region_2024")
    else:
        in_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "region_2024")

**Load SSB 10km Grid into DuckDB**

In [29]:
# import SSB 10km grid

if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "norge_ruter_10km")
    else:
        in_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "norge_ruter_10km")

### Preprocess DuckDB Tables 

**Convert BLOB columns to GEOM columns**

In [30]:
if prepare_duckdb:
    # cast BLOB (Binary Large OBject) to geometry for spatial operations
    
    import duckdb

    con = duckdb.connect(db_path)
    con.install_extension('spatial')
    con.load_extension('spatial')
    
    # duckdb tables names to list
    tables = con.execute("SHOW TABLES;").fetchdf()
    tables = tables["name"].to_list()
    print(tables)
    
    for table in tables:
        
        # TODO add check if geom field exists
        
        blob_to_geom(
            db_path=db_path,
            tbl_name=table,
            blob_field="geometry",
            geom_field="geom",   
        )

### Display DuckDB tables 

In [31]:
import duckdb

con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

In [32]:
# print tables
con.sql("SHOW TABLES;")

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ ar50_flate       │
│ foreslatt_vern   │
│ naturvernomrade  │
│ norge_ruter_10km │
│ region_2024      │
│ soner_2017_1km   │
│ vern_land        │
│ vern_land_sjo    │
│ vern_land_tmp1   │
│ vern_land_tmp2   │
│ vern_land_vars   │
│ vern_sjo         │
│ vern_sjo_tmp1    │
│ vern_sjo_tmp2    │
├──────────────────┤
│     14 rows      │
└──────────────────┘

In [33]:
# display first 5 rows of the table
table = "foreslatt_vern"
con.sql(f"SELECT * FROM {table} LIMIT 5")

┌──────────────────────┬──────────────────────┬────────────┬───┬───────────────┬───────────┬──────────────────────┐
│ identifikasjon_lok…  │ identifikasjon_nav…  │    navn    │ … │   verneform   │ verneplan │         geom         │
│       varchar        │       varchar        │  varchar   │   │    varchar    │  varchar  │       geometry       │
├──────────────────────┼──────────────────────┼────────────┼───┼───────────────┼───────────┼──────────────────────┤
│ VP00000313           │ http://www.miljodi…  │ Inngripan  │ … │ naturreservat │           │ MULTIPOLYGON (((13…  │
│ VP00000404           │ http://www.miljodi…  │ Kverndalen │ … │ naturreservat │           │ MULTIPOLYGON (((34…  │
│ VP00000410           │ http://www.miljodi…  │ Båsdalen   │ … │ naturreservat │           │ MULTIPOLYGON (((41…  │
│ VP00000416           │ http://www.miljodi…  │ Rennen     │ … │ naturreservat │           │ MULTIPOLYGON (((29…  │
│ VP00000418           │ http://www.miljodi…  │ Leksa      │ … │ naturre

In [34]:
# close
con.close()

## Display Data on the Map

https://ipyleaflet.readthedocs.io/en/latest/installation/index.html#using-pip 


In [35]:
# Display data on map using leafmap
# TODO display duckdb tables as well, not only WMS

init_location = [62.223207, 9.550195]  # Hjerkinn
zoom_start = 13  
min_zoom = 8  # keeps user from zooming out too far
basemap = leafmap.Map(
    location=init_location, 
    zoom=zoom_start, 
    min_zoom=min_zoom, 
    max_bounds=True
    )

# set background
basemap.add_basemap("SATELLITE", opacity=0.7)

# add WMS verneområder
wms_url ="https://kart.miljodirektoratet.no/arcgis/services/vern/mapserver/WMSServer"
wms_layer = "naturvern_klasser_omrade"
wms_name = "Protected Areas"
basemap.add_wms_layer(
    url=wms_url, 
    layers=wms_layer, 
    name=wms_name, 
    wms_format="image/png",
    )

if test_area:
    # add bounding box - Hjerkinn
    basemap.add_gdf(gdf_BB, layer_name="Test Area", fill_color="blue", fill_opacity=0.2, weight=2)

basemap

# Methods


### 1. Overlap protected areas and Bioclimatic Zones
**Bioklimatiske soner**

| Protected area | Sone 1 (%) | Sone 2 (%) | Sone 3 (%) | ... |
|----------------|------------|------------|------------|-----|
| *NaturvernId*  |*bioklima_1*|*bioklima_2*|*bioklima_3*|...  |
| area A         |            |            |            |     |

In [36]:
if run_bioklima:
    tbl_study_area = protected_area
    id = "identifikasjon_lokalId"
    tbl_bioklima = "soner_2017_1km"
    bioklima_field = "Sone_Kode"
    bioklima_zones = ["6SO-1", "6SO-2", "6SO-3", "6SO-4", "6SO-5"]
    for area_class in bioklima_zones: # 1-18
        print(f"Calculating Bioklima class {area_class}")
        new_field = f"s_{area_class}" # ar50_bon1
        new_field = new_field.replace("-", "_")
        remove_field(db_path, tbl_study_area, new_field)
        bioklima_area_class(db_path, tbl_study_area, id, tbl_bioklima, bioklima_field, area_class, new_field)
    
    for area_class in bioklima_zones: # 1-18
        new_field = f"s_{area_class}" # ar50_bon1
        new_field = new_field.replace("-", "_")
        con = duckdb.connect(db_path)
        con.install_extension('spatial')
        con.load_extension('spatial')
        con.sql(f"UPDATE {protected_area} SET {new_field} = 0 WHERE {new_field} IS NULL")
        con.close()

In [37]:
con = duckdb.connect(db_path)
con.sql(f"SELECT * FROM {protected_area} LIMIT 5")

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬───────────┬───────────┬────────────┐
│       faktaark       │         uuid         │ identifikasjon_lok…  │ … │ areal_m2  │ omkrets_m │ formindeks │
│       varchar        │       varchar        │       varchar        │   │   float   │   float   │   float    │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼───────────┼───────────┼────────────┤
│ https://faktaark.n…  │ {1E57ECC6-4017-448…  │ VV00000825           │ … │  66602.15 │ 1313.6952 │  1.4359707 │
│ https://faktaark.n…  │ {1E4A3735-13E8-4AC…  │ VV00001473           │ … │  9547.154 │ 484.12164 │  1.3976957 │
│ https://faktaark.n…  │ {ABCA04BD-8410-474…  │ VV00001465           │ … │  94993.68 │ 1289.1515 │  1.1799176 │
│ https://faktaark.n…  │ {9299F6EB-AA11-490…  │ VV00001758           │ … │  32635.92 │ 719.23157 │  1.1230934 │
│ https://faktaark.n…  │ {639D89AA-BFD3-4F5…  │ VV00001054           │ … │ 4358982.0 │ 13253.623 │   1.7

### 2. Overlap protected areas and AR50 (land cover)

**Divide the protected areas into terrestrial and marine areas**

AR50 polygons with classified as "Hav" are considered marine areas, all other areas are considered terrestrial.
- Bonitet = 17 (Hav) 
- Arealtype = 82 (Hav)

<br>

**AR50 - Bonitet**
Calculated area overlap with AR50 Bonitet classes. 

| Protected area | Fulldyrka og overflatedyrka jord (m2) | Innmarksbeite (m2) | Skog, høg og særs høg bonitet (m2) | ... |
|----------------|--------------------------------------|-------------------|-----------------------------------|-----|
| *NaturvernId*  |*AR50_bon_1*|*AR50_bon2*|*AR50_bon3*|...|
| area A         |                                      |                   |                                   |     |


In [38]:
# calculate ar50 area overlapp with protected areas
tbl_study_area = protected_area
id = "identifikasjon_lokalId"
tbl_ar50 = "ar50_flate"
ar50_field = "ar50_bonitet"

if run_ar50:
    for area_class in range(1, 19): # 1-18
        print(f"Calculating Bonitet class {area_class}")
        new_field = f"ar50_bon{area_class}" # ar50_bon1
        remove_field(db_path, tbl_study_area, new_field)
        ar50_area_class(db_path, tbl_study_area, id, tbl_ar50, ar50_field, area_class, new_field)

    # if ar50_bonx is null, set to 0
    for area_class in range(1, 19): # 1-18
        new_field = f"ar50_bon{area_class}" # ar50_bon1
        con = duckdb.connect(db_path)
        con.install_extension('spatial')
        con.load_extension('spatial')
        con.sql(f"UPDATE {protected_area} SET {new_field} = 0 WHERE {new_field} IS NULL")
        con.close()

In [39]:
# print first 5 rows of the table protected_area
con = duckdb.connect(db_path)
con.sql(f"SELECT * FROM {protected_area} LIMIT 5")

# print as dataframe
df = con.execute(f"SELECT * FROM {protected_area} LIMIT 5").fetchdf()

# display columns: identfifikasjon_lokalId, ar50_bonx
col_id= [id]
cols_ar50 = [col for col in df.columns if "ar50_bon" in col and col != "sum_ar50_bon_m2"]
cols = col_id + cols_ar50
df_ar50 = df[cols]
display(df)

# close
con.close()

,faktaark,uuid,identifikasjon_lokalId,identifikasjon_navnerom,kommune,forvaltningsmyndighet,forvaltningsmyndighetType,cddaId,førstegangVernet,forvaltningsplan,...,ar50_bon14,ar50_bon15,ar50_bon16,ar50_bon17,ar50_bon18,sum_ar50_bon_m2,area_diff_m2,areal_m2,omkrets_m,formindeks
0,https://faktaark.naturbase.no/?id=VV00000825,{1E57ECC6-4017-448B-86BE-4A8424DD8BCF},VV00000825,https://www.miljodirektoratet.no/naturbase,1112,Statsforvalteren i Rogaland,statsforvalter,155680,None,ingenPlan,...,0.0,0.0,0.000,0.000000,0.0,6.660215e+04,0.000000,6.660215e+04,1313.695190,1.435971
1,https://faktaark.naturbase.no/?id=VV00001473,{1E4A3735-13E8-4AC3-92F7-CEE94FB95BCD},VV00001473,https://www.miljodirektoratet.no/naturbase,4226,Hægebostad kommune,kommune,155566,None,ingenPlan,...,0.0,0.0,0.000,0.000000,0.0,9.547154e+03,0.000000,9.547154e+03,484.121643,1.397696
2,https://faktaark.naturbase.no/?id=VV00001465,{ABCA04BD-8410-474E-B0CE-FF484E58AB73},VV00001465,https://www.miljodirektoratet.no/naturbase,4205,Vernområdestyret for Oksøy-Ryvingen og Flekkef...,verneområdestyre,156300,None,revideres,...,0.0,0.0,0.000,47219.203125,0.0,9.499367e+04,-0.007812,9.499368e+04,1289.151489,1.179918
3,https://faktaark.naturbase.no/?id=VV00001758,{9299F6EB-AA11-4902-A946-AB3D8C96A95B},VV00001758,https://www.miljodirektoratet.no/naturbase,4631,Statsforvalteren i Vestland,statsforvalter,156165,None,godkjent,...,0.0,0.0,0.000,32635.919922,0.0,3.263592e+04,0.000000,3.263592e+04,719.231567,1.123093
4,https://faktaark.naturbase.no/?id=VV00001054,{639D89AA-BFD3-4F58-9A63-457DCF1169E1},VV00001054,https://www.miljodirektoratet.no/naturbase,4619,Statsforvalteren i Vestland,statsforvalter,156408,None,ingenPlan,...,0.0,0.0,290855.625,0.000000,0.0,4.358982e+06,0.000000,4.358982e+06,13253.623047,1.790759


**Check if sum of all AR50 classes correspons with the total area**

In [40]:
# Calculating sum of all bonitet classes
remove_field(db_path, tbl_study_area, "sum_ar50_bon_m2")
sum_area_cols(db_path, tbl_study_area, id, cols_ar50, "sum_ar50_bon_m2")

# Calculating area of each polygon in the study area
remove_field(db_path, tbl_study_area, "areal_m2")
geom_area(db_path, tbl_study_area, "geom", "areal_m2")

# Calculateing the difference between the sum of all bonitet classes and the area of each polygon
remove_field(db_path, tbl_study_area, "area_diff_m2")
area_difference(db_path, tbl_study_area, "sum_ar50_bon_m2", "areal_m2", "area_diff_m2")

In [41]:
# print first 5 rows of the table protected_area
con = duckdb.connect(db_path)

# print as dataframe
df = con.execute(f"SELECT * FROM {protected_area}").fetchdf()

# display columns: identfifikasjon_lokalId, ar50_bonx
col_id= [id]
cols = col_id + ["navn", "sum_ar50_bon_m2", "areal_m2", "area_diff_m2"]
df_ar50 = df[cols]

# sort by area_diff_m2
df_ar50 = df_ar50.sort_values(by="area_diff_m2", ascending=True)
display(df_ar50)

# close
con.close()

,identifikasjon_lokalId,navn,sum_ar50_bon_m2,areal_m2,area_diff_m2
1290,VV00002567,Nordaust-Svalbard,0.000000e+00,5.535372e+10,-5.535372e+10
427,VV00002568,Søraust-Svalbard,0.000000e+00,2.182590e+10,-2.182590e+10
1574,VV00002565,Sør-Spitsbergen,0.000000e+00,1.317920e+10,-1.317920e+10
359,VV00002573,Nordvest-Spitsbergen,0.000000e+00,9.841827e+09,-9.841827e+09
1222,VV00003010,Jan Mayen,0.000000e+00,4.759954e+09,-4.759954e+09
...,...,...,...,...,...
2694,VV00000252,Junkerdal,6.820593e+08,6.820591e+08,1.280000e+02
2508,VV00002750,Lomsdal-Visten,1.101799e+09,1.101799e+09,1.280000e+02
3247,VV00000235,Saltfjellet-Svartisen,2.192003e+09,2.192003e+09,2.560000e+02
2034,VV00001878,Hardangervidda,3.444758e+09,3.444758e+09,2.560000e+02


In [42]:
# print number of rows 
print(f"Number of rows: {df.shape[0]}")

# print number of unique ids "identifikasjon_lokalId"
print(f"Number of unique ids: {df['identifikasjon_lokalId'].nunique()}")

# print number of rows with value in "ar50_bon17"
print(f"Number of areas with Marine area: {df['ar50_bon17'].count()}")

Number of rows: 3370
Number of unique ids: 3370
Number of areas with Marine area: 3370


### 2. Geometry variables for protected areas

| Protected area | Total area (km2) | Perimeter (km) | Land area (km2) | Perimeter land area (km) |
|----------------|------------|----------------|-----------------|-------------------------|
| *NaturvernId* | *areal_m2* | *omkrets_m* | *landareal_m2* | *landomkrets_m2*|
| area A         |    *        |                |                 |                         |

***
$\mathbf{\text{Shape Index}}$<br>
***
The shape index is a measure of how compact the shape is compared to a circle with the same area. The shape index is calculated for the entire protected area and the land area.

- $P$ = perimeter
- $r$ = radius
- $A$ = area


**Shape Index:**&emsp;      $SI = \frac{P}{2\pi r}$

**Radius:**&emsp; $r = \sqrt{\frac{A}{\pi}}$

- SI = 1, shape is a perfect circle
- SI > 1, shape is less compact than a circle
- SI < 1, is not possible 

In [43]:
if run_shape_index:
    # create two new tables with only terrestrial part and marine part, respectively.
    remove_table(db_path, protected_land)
    remove_table(db_path, protected_sea)
    remove_table(db_path, f"{protected_land}_tmp1")
    remove_table(db_path, f"{protected_sea}_tmp1")
    extract_overlap_geom(db_path, id, tbl_ar50, "ar50_bonitet", 17, tbl_study_area, f"{protected_sea}_tmp1", f"{protected_land}_tmp1")

In [44]:
if run_shape_index:

    # remove slivers (lines/points) from the tables
    remove_table(db_path, f"{protected_land}_tmp2")
    remove_table(db_path, f"{protected_sea}_tmp2")
    delete_lines_points(db_path, f"{protected_land}_tmp1", f"{protected_land}_tmp2")
    delete_lines_points(db_path, f"{protected_sea}_tmp1", f"{protected_sea}_tmp2")

    # to multipolygon
    remove_table(db_path, protected_land)
    remove_table(db_path, protected_sea)
    group_to_multipolygon(db_path, f"{protected_land}_tmp2", protected_land, id)
    group_to_multipolygon(db_path, f"{protected_sea}_tmp2", protected_sea, id)

In [45]:
dict_geom = {
    protected_area: ("areal_m2", "omkrets_m", "formindeks"),
    protected_land: ("landareal_m2", "landomkrets_m", "land_formindeks"),
    protected_sea: ("sjoareal_m2", "sjoomkrets_m", "sjo_formindeks"),
}

for key, value in dict_geom.items():
    
    # areal
    #remove_field(db_path, key, "areal_not_grouped")
    #geom_area(db_path, key, "geom", "areal_not_grouped")
    
    # areal by ID
    remove_field(db_path, key, value[0])
    geom_area_byID(db_path, key, "identifikasjon_lokalId", "geom", value[0])
    
    # omkrets
    #remove_field(db_path, key, "omkrets_not_grouped")
    #geom_peri(db_path, key, "geom", "omkrets_not_grouped")
    
    # omkrets by ID
    remove_field(db_path, key, value[1])
    geom_peri_byID(db_path, key, "identifikasjon_lokalId", "geom", value[1])
    
    # formindeks
    #remove_field(db_path, key, "formindeks_not_grouped")
    #geom_index(db_path, key, "geom", "formindeks_not_grouped")
    
    # formindeks by ID
    remove_field(db_path, key, value[2])
    geom_index_byID(db_path, key, "identifikasjon_lokalId", "geom", value[2])


An error occurred: Binder Error: Referenced column "geom" not found in FROM clause!
Candidate bindings: "vern_land.landareal_m2"
LINE 5: ...vern_land as study_area
                SET landareal_m2 = subquery.area
                FROM (
                    SELECT identifikasjon_lokalId, SUM(ST_Area(geom)) as area
                                                  ^
An error occurred: Binder Error: Referenced column "geom" not found in FROM clause!
Candidate bindings: "vern_land.landareal_m2"
LINE 5: ...land as study_area
                SET landomkrets_m = subquery.perimeter
                FROM (
                    SELECT identifikasjon_lokalId, SUM(ST_Perimeter(geom)) as perimeter
                                                  ^
An error occurred: Binder Error: Referenced column "geom" not found in FROM clause!
Candidate bindings: "vern_land.landareal_m2"
LINE 5: ...land as study_area
                SET land_formindeks = subquery.perimeter / ...
                                   

In [46]:
con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

In [47]:
con.sql("SHOW TABLES;")

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ ar50_flate       │
│ foreslatt_vern   │
│ naturvernomrade  │
│ norge_ruter_10km │
│ region_2024      │
│ soner_2017_1km   │
│ vern_land        │
│ vern_land_sjo    │
│ vern_land_tmp1   │
│ vern_land_tmp2   │
│ vern_land_vars   │
│ vern_sjo         │
│ vern_sjo_tmp1    │
│ vern_sjo_tmp2    │
├──────────────────┤
│     14 rows      │
└──────────────────┘

In [48]:
con.sql(f"SELECT * FROM {protected_land} LIMIT 5")

┌────────────────────────┬──────────────┬───────────────┬─────────────────┐
│ identifikasjon_lokalId │ landareal_m2 │ landomkrets_m │ land_formindeks │
│        varchar         │    float     │     float     │      float      │
├────────────────────────┼──────────────┼───────────────┼─────────────────┤
│ VV00000104             │         NULL │          NULL │            NULL │
│ VV00003065             │         NULL │          NULL │            NULL │
│ VV00001059             │         NULL │          NULL │            NULL │
│ VV00003822             │         NULL │          NULL │            NULL │
│ VV00003447             │         NULL │          NULL │            NULL │
└────────────────────────┴──────────────┴───────────────┴─────────────────┘

In [49]:
con.sql(f"SELECT * FROM {protected_sea} LIMIT 5")

┌────────────────────────┬─────────────┬──────────────┬────────────────┐
│ identifikasjon_lokalId │ sjoareal_m2 │ sjoomkrets_m │ sjo_formindeks │
│        varchar         │    float    │    float     │     float      │
├────────────────────────┼─────────────┼──────────────┼────────────────┤
│ VV00000289             │        NULL │         NULL │           NULL │
│ VV00002312             │        NULL │         NULL │           NULL │
│ VV00002324             │        NULL │         NULL │           NULL │
│ VV00000280             │        NULL │         NULL │           NULL │
│ VV00000293             │        NULL │         NULL │           NULL │
└────────────────────────┴─────────────┴──────────────┴────────────────┘

**Export to GDF and GPKG-layer**

In [50]:
# to dataframe
df_land = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {protected_land}").fetchdf()

# drop "geom"
df_land = df_land.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df_land['geometry'] = df_land['geometry'].apply(wkt.loads)

# to gpd
gdf_land = gpd.GeoDataFrame(df_land, geometry='geometry')
print(f"Number of unique ids: {df_land['identifikasjon_lokalId'].nunique()}")

display(gdf_land.head())

# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf_land.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_land.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=protected_land, mode='w')


BinderException: Binder Error: Referenced column "geom" not found in FROM clause!
Candidate bindings: "vern_land.landareal_m2"
LINE 1: SELECT ST_AsText(geom) as geometry, * FROM vern_land
                         ^

In [ ]:
# to dataframe
df_sjo = con.execute(f"""
    SELECT ST_AsText(geom) as geometry, * 
    FROM {protected_sea}
    WHERE ST_GeometryType(geom) IN ('POLYGON', 'MULTIPOLYGON')
""").fetchdf()

# delete POLYGON EMPTY
df_sjo = df_sjo[df_sjo['geometry'] != "POLYGON EMPTY"]

# drop "geom"
df_sjo = df_sjo.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df_sjo['geometry'] = df_sjo['geometry'].apply(wkt.loads)

# to gpd
gdf_sjo = gpd.GeoDataFrame(df_sjo, geometry='geometry')
print(f"Number of unique ids: {gdf_sjo['identifikasjon_lokalId'].nunique()}")

display(gdf_sjo)

# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf_sjo.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_sjo.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=protected_sea, mode='w')

In [ ]:
# drop geometry column in duckdb and keep only unique ids in table

# drop geom column
remove_field(db_path, protected_land, "geom")
remove_field(db_path, protected_sea, "geom")

# drop duplicates 
remove_duplicates(db_path, protected_land, "identifikasjon_lokalId")
remove_duplicates(db_path, protected_sea, "identifikasjon_lokalId")

In [ ]:
# number of rows in the new table
con.sql(f"SELECT COUNT(*) FROM {protected_land}")

In [ ]:
# number of unique ids in the new table
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {protected_land}")

In [ ]:
remove_table(db_path, protected_land_sea)
join_tables_create_new(
    db_path, 
    tbl1=protected_land,
    tbl2=protected_sea,
    id_field="identifikasjon_lokalId",
    new_tbl=protected_land_sea
)

remove_field(db_path, protected_land_sea, "identifikasjon_lokalId_1") 
con.sql("SHOW TABLES;")

In [ ]:
con.sql(f"SELECT * FROM {protected_land_sea} LIMIT 5")

In [ ]:
# number of rows in the new table
con.sql(f"SELECT COUNT(*) FROM {protected_land_sea}")

In [ ]:
# number of unique ids in the new table
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {protected_land_sea}")

In [ ]:
remove_table(db_path, protected_land_vars)
join_tables_create_new(
    db_path, 
    tbl1=protected_area,
    tbl2=protected_land_sea,
    id_field="identifikasjon_lokalId",
    new_tbl=protected_land_vars
)

In [ ]:
con.sql("SHOW TABLES;")

In [ ]:
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {protected_land_vars}")

**Clean table**

In [ ]:
# if sjoareal is 95% of the total area, then it is a marine area
# set sjoareal, omkrets and index to 0 if it is less than 5% of the total area or where sjoareal is NULL
# set landareal, omkrets and index  to 0 if it is less than 5% of the total area

# update table 
#con.sql(f"""
#    UPDATE {protected_land_vars}
#    SET sjoareal_m2 = 0, sjoomkrets_m = 0
#    WHERE sjoareal_m2 < 0.05 * areal_m2
#""")

# update table
#con.sql(f"""
#    UPDATE {protected_land_vars}
#    SET landareal_m2 = 0, landomkrets_m = 0
#    WHERE landareal_m2 < 0.05 * areal_m2
#""")

# if null set to 0
con.sql(f"""
    UPDATE {protected_land_vars}
    SET sjoareal_m2 = 0, sjoomkrets_m = 0
    WHERE sjoareal_m2 IS NULL
""")

# if null set to 0
con.sql(f"""
    UPDATE {protected_land_vars}
    SET landareal_m2 = 0, landomkrets_m = 0
    WHERE landareal_m2 IS NULL
""")

# remove cols
remove_field(db_path, protected_land_vars, "identifikasjon_lokalId_1")


# print first 5 rows of the table protected_land_vars
con.sql(f"SELECT * FROM {protected_land_vars} LIMIT 5")

In [53]:
# to dataframe
df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {protected_land_vars}").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf['identifikasjon_lokalId'].nunique()}")


display(gdf)

Number of unique ids: 3370


,geometry,faktaark,uuid,identifikasjon_lokalId,identifikasjon_navnerom,kommune,forvaltningsmyndighet,forvaltningsmyndighetType,cddaId,førstegangVernet,...,area_diff_m2,areal_m2,omkrets_m,formindeks,landareal_m2,landomkrets_m,land_formindeks,sjoareal_m2,sjoomkrets_m,sjo_formindeks
0,"MULTIPOLYGON (((6643.780 6506797.010, 6783.060...",https://faktaark.naturbase.no/?id=VV00000825,{1E57ECC6-4017-448B-86BE-4A8424DD8BCF},VV00000825,https://www.miljodirektoratet.no/naturbase,1112,Statsforvalteren i Rogaland,statsforvalter,155680,None,...,0.000000,6.660215e+04,1313.695190,1.435971,6.660215e+04,1313.695190,1.435971,0.000000,0.000000,NaN
1,"MULTIPOLYGON (((45864.870 6509569.200, 45892.1...",https://faktaark.naturbase.no/?id=VV00001473,{1E4A3735-13E8-4AC3-92F7-CEE94FB95BCD},VV00001473,https://www.miljodirektoratet.no/naturbase,4226,Hægebostad kommune,kommune,155566,None,...,0.000000,9.547154e+03,484.121643,1.397696,9.547154e+03,484.121643,1.397696,0.000000,0.000000,NaN
2,"MULTIPOLYGON (((59675.230 6449318.120, 59608.8...",https://faktaark.naturbase.no/?id=VV00001465,{ABCA04BD-8410-474E-B0CE-FF484E58AB73},VV00001465,https://www.miljodirektoratet.no/naturbase,4205,Vernområdestyret for Oksøy-Ryvingen og Flekkef...,verneområdestyre,156300,None,...,-0.007812,9.499368e+04,1289.151489,1.179918,4.777447e+04,1171.103516,1.511445,47219.203125,1900.927490,2.467751
3,"MULTIPOLYGON (((82453.430 6716602.870, 82432.0...",https://faktaark.naturbase.no/?id=VV00001054,{639D89AA-BFD3-4F58-9A63-457DCF1169E1},VV00001054,https://www.miljodirektoratet.no/naturbase,4619,Statsforvalteren i Vestland,statsforvalter,156408,None,...,0.000000,4.358982e+06,13253.623047,1.790759,4.358982e+06,13253.623047,1.790759,0.000000,0.000000,NaN
4,"MULTIPOLYGON (((487370.460 7418945.410, 487794...",https://faktaark.naturbase.no/?id=VV00003027,{83C0FF85-F7EF-4869-9B97-EE96F8B80883},VV00003027,https://www.miljodirektoratet.no/naturbase,1839,Statsforvalteren i Nordland,statsforvalter,555545899,None,...,-0.062500,6.609629e+05,3142.556396,1.090410,6.609629e+05,3142.556396,1.090410,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,11366.940430,400.250702,1.059023
3566,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,17342.189453,515.478760,1.104216
3567,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,21310.105469,530.084961,1.024348
3568,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,18780.183594,511.252655,1.052400


In [54]:
# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf.crs = "EPSG:25833"
# Write to existing .geopackage
gdf.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=protected_land_vars, mode='w')

# Write to .csv
gdf.to_csv(os.path.join(filepath + '.csv'))

**Infrastrukturindeks**

| Protected area | Mean value | Lav (%) | Middels (%) | Høy (%) | ... |
|----------------|------------|---------|-------------|---------|-----|
| area A         |            |         |             |         |     |


In [55]:
# drop all rows where id  is NULL
gdf = gdf.dropna(subset=["identifikasjon_lokalId"])

In [56]:
display(gdf)

,geometry,faktaark,uuid,identifikasjon_lokalId,identifikasjon_navnerom,kommune,forvaltningsmyndighet,forvaltningsmyndighetType,cddaId,førstegangVernet,...,area_diff_m2,areal_m2,omkrets_m,formindeks,landareal_m2,landomkrets_m,land_formindeks,sjoareal_m2,sjoomkrets_m,sjo_formindeks
0,"MULTIPOLYGON (((6643.780 6506797.010, 6783.060...",https://faktaark.naturbase.no/?id=VV00000825,{1E57ECC6-4017-448B-86BE-4A8424DD8BCF},VV00000825,https://www.miljodirektoratet.no/naturbase,1112,Statsforvalteren i Rogaland,statsforvalter,155680,None,...,0.000000,6.660215e+04,1313.695190,1.435971,6.660215e+04,1313.695190,1.435971,0.000000,0.000000,NaN
1,"MULTIPOLYGON (((45864.870 6509569.200, 45892.1...",https://faktaark.naturbase.no/?id=VV00001473,{1E4A3735-13E8-4AC3-92F7-CEE94FB95BCD},VV00001473,https://www.miljodirektoratet.no/naturbase,4226,Hægebostad kommune,kommune,155566,None,...,0.000000,9.547154e+03,484.121643,1.397696,9.547154e+03,484.121643,1.397696,0.000000,0.000000,NaN
2,"MULTIPOLYGON (((59675.230 6449318.120, 59608.8...",https://faktaark.naturbase.no/?id=VV00001465,{ABCA04BD-8410-474E-B0CE-FF484E58AB73},VV00001465,https://www.miljodirektoratet.no/naturbase,4205,Vernområdestyret for Oksøy-Ryvingen og Flekkef...,verneområdestyre,156300,None,...,-0.007812,9.499368e+04,1289.151489,1.179918,4.777447e+04,1171.103516,1.511445,47219.203125,1900.927490,2.467751
3,"MULTIPOLYGON (((82453.430 6716602.870, 82432.0...",https://faktaark.naturbase.no/?id=VV00001054,{639D89AA-BFD3-4F58-9A63-457DCF1169E1},VV00001054,https://www.miljodirektoratet.no/naturbase,4619,Statsforvalteren i Vestland,statsforvalter,156408,None,...,0.000000,4.358982e+06,13253.623047,1.790759,4.358982e+06,13253.623047,1.790759,0.000000,0.000000,NaN
4,"MULTIPOLYGON (((487370.460 7418945.410, 487794...",https://faktaark.naturbase.no/?id=VV00003027,{83C0FF85-F7EF-4869-9B97-EE96F8B80883},VV00003027,https://www.miljodirektoratet.no/naturbase,1839,Statsforvalteren i Nordland,statsforvalter,555545899,None,...,-0.062500,6.609629e+05,3142.556396,1.090410,6.609629e+05,3142.556396,1.090410,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3365,"MULTIPOLYGON (((265884.010 6678788.710, 265880...",https://faktaark.naturbase.no/?id=VV00003422,{6486DEBE-8313-4E78-A6B9-193DC06CF493},VV00003422,https://www.miljodirektoratet.no/naturbase,3234,Statsforvalteren i Oslo og Viken,statsforvalter,555639858,None,...,-0.250000,3.752726e+06,10527.068359,1.532954,3.752726e+06,10527.068359,1.532954,0.000000,0.000000,NaN
3366,"MULTIPOLYGON (((1019436.890 7917039.670, 10190...",https://faktaark.naturbase.no/?id=VV00000053,{4D508E1E-F875-495E-AD5A-2278BAF9C0B4},VV00000053,https://www.miljodirektoratet.no/naturbase,5630,Statsforvalteren i Troms og Finnmark,statsforvalter,156026,None,...,0.000000,4.837579e+06,8927.736328,1.145045,4.454772e+06,10888.909180,1.455348,382806.968750,4109.854492,1.873836
3367,"MULTIPOLYGON (((451663.520 7274595.870, 451370...",https://faktaark.naturbase.no/?id=VV00003299,{43D74F43-5B7D-42AA-A80A-BD5765ACEBC7},VV00003299,https://www.miljodirektoratet.no/naturbase,"1826,1825",Statsforvalteren i Nordland,statsforvalter,555634147,None,...,-1.000000,1.608238e+07,45223.402344,3.181143,1.608238e+07,45223.402344,3.181143,0.000000,0.000000,NaN
3368,"MULTIPOLYGON (((280229.600 6664857.850, 280229...",https://faktaark.naturbase.no/?id=VV00002102,{73354DCD-E1CC-4CC1-AC22-59D8FD33A116},VV00002102,https://www.miljodirektoratet.no/naturbase,3230,Statsforvalteren i Oslo og Viken,statsforvalter,183343,None,...,0.000000,3.128960e+02,62.770050,1.001031,3.128960e+02,62.770050,1.001031,0.000000,0.000000,NaN


In [57]:
from py_scripts.raster import * 

# calculate the min, max, mean, std and median of the infrastruktur raster 
# within the protected areas with a buffer of 1000m
buffer = 1000
raster_path = os.path.join(interim_data, "infra_25m_25833_clip.tif")

gdf_infra = overlay_stats(raster_path, gdf, buffer_distance=1000)


In [58]:
# Export GDF to file
gdf_infra.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_infra.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=f"{protected_land_vars}_infra", mode='w')



### 3. Regional Statistics


Administrative regions:
- Municipalities
- Counties
- Regions 
    - **Nord**: Finnmark, Troms, Nordland
    - **Midt**: Trøndelag, Møre og Romsdal
    - **Sør**: Agder, Vestfold, Telemark
    - **Øst**: Østfold, Akershus, Oslo, Innlandet, Buskerud
    - **Vest**: Vestland, Rogaland


| Protected area | Municipality | County | Region |
|----------------|---------|-------|--------|
| *NaturvernId*  |*kommune*|*fylke*|*region*|
| area A         |         |       |        |

*Regional datasett*:

| Region | Area | Protected Area | Land Protected Area | Marine Protected Area |
|--------|------|----------------|---------------------|-----------------------|
| *Region* | *areal* | *a_vern* | *a_landvern* | *a_marinvern* |
| Nord ||||||


In [59]:
# DuckDB to dataframe
df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {protected_land_vars}").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf_protected_area = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf_protected_area['identifikasjon_lokalId'].nunique()}")


display(gdf_protected_area)

Number of unique ids: 3370


,geometry,faktaark,uuid,identifikasjon_lokalId,identifikasjon_navnerom,kommune,forvaltningsmyndighet,forvaltningsmyndighetType,cddaId,førstegangVernet,...,area_diff_m2,areal_m2,omkrets_m,formindeks,landareal_m2,landomkrets_m,land_formindeks,sjoareal_m2,sjoomkrets_m,sjo_formindeks
0,"MULTIPOLYGON (((6643.780 6506797.010, 6783.060...",https://faktaark.naturbase.no/?id=VV00000825,{1E57ECC6-4017-448B-86BE-4A8424DD8BCF},VV00000825,https://www.miljodirektoratet.no/naturbase,1112,Statsforvalteren i Rogaland,statsforvalter,155680,None,...,0.000000,6.660215e+04,1313.695190,1.435971,6.660215e+04,1313.695190,1.435971,0.000000,0.000000,NaN
1,"MULTIPOLYGON (((45864.870 6509569.200, 45892.1...",https://faktaark.naturbase.no/?id=VV00001473,{1E4A3735-13E8-4AC3-92F7-CEE94FB95BCD},VV00001473,https://www.miljodirektoratet.no/naturbase,4226,Hægebostad kommune,kommune,155566,None,...,0.000000,9.547154e+03,484.121643,1.397696,9.547154e+03,484.121643,1.397696,0.000000,0.000000,NaN
2,"MULTIPOLYGON (((59675.230 6449318.120, 59608.8...",https://faktaark.naturbase.no/?id=VV00001465,{ABCA04BD-8410-474E-B0CE-FF484E58AB73},VV00001465,https://www.miljodirektoratet.no/naturbase,4205,Vernområdestyret for Oksøy-Ryvingen og Flekkef...,verneområdestyre,156300,None,...,-0.007812,9.499368e+04,1289.151489,1.179918,4.777447e+04,1171.103516,1.511445,47219.203125,1900.927490,2.467751
3,"MULTIPOLYGON (((82453.430 6716602.870, 82432.0...",https://faktaark.naturbase.no/?id=VV00001054,{639D89AA-BFD3-4F58-9A63-457DCF1169E1},VV00001054,https://www.miljodirektoratet.no/naturbase,4619,Statsforvalteren i Vestland,statsforvalter,156408,None,...,0.000000,4.358982e+06,13253.623047,1.790759,4.358982e+06,13253.623047,1.790759,0.000000,0.000000,NaN
4,"MULTIPOLYGON (((487370.460 7418945.410, 487794...",https://faktaark.naturbase.no/?id=VV00003027,{83C0FF85-F7EF-4869-9B97-EE96F8B80883},VV00003027,https://www.miljodirektoratet.no/naturbase,1839,Statsforvalteren i Nordland,statsforvalter,555545899,None,...,-0.062500,6.609629e+05,3142.556396,1.090410,6.609629e+05,3142.556396,1.090410,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,11366.940430,400.250702,1.059023
3566,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,17342.189453,515.478760,1.104216
3567,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,21310.105469,530.084961,1.024348
3568,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,18780.183594,511.252655,1.052400


In [60]:
# DuckDB to dataframe
df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM regions").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf_region = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf_region['identifikasjon_lokalId'].nunique()}")

display(gdf_region)

CatalogException: Catalog Error: Table with name regions does not exist!
Did you mean "region_2024"?
LINE 1: ...CT ST_AsText(geom) as geometry, * FROM regions
                                                  ^

In [ ]:
import geopandas as gpd

def sum_protected_area_in_region(gdf_region, gdf_protected_area):
    # Perform a spatial join of the region and protected area GeoDataFrames
    joined = gpd.sjoin(gdf_region, gdf_protected_area, how="inner", op='intersects')

    # Group by the region geometry and sum the protected area
    area_sum = joined.groupby(joined.geometry)['areal_m2'].sum()

    # Reset the index of the region GeoDataFrame to its geometry
    gdf_region.set_index(gdf_region.geometry, inplace=True)

    # Add the summed area as a new column to the region GeoDataFrame
    gdf_region['total_protected_area'] = area_sum

    # Reset the index of the region GeoDataFrame
    gdf_region.reset_index(drop=True, inplace=True)

    return gdf_region

###  4. Grid Statistics (10x10 km2)

**Rutenettstatistikk:**

| SSB grid cell | area | Density (land + marine) | Density (land) | 
| --------------|------|-------------------------|----------------|
| *grid_ID* | *grid_areal* | *tetthet_tot_vern* | *tetthet_landvern* | 
| cell 1| 100 km2 | ||





***
$\mathbf{\text{Density}}$<br>
***
The density of protected area per 10x10km (SSB grid) is calculated by dividing the area of the protected area by 100 km^2.

**Density:**&emsp;      $Density = \frac{A}{100 km^2}$



In [ ]:
# Select by region
# SUM protected area in region 
# Area % = SUM(protected_area_m2)/(areal_m2) * 100